# Saving and Loading rule bases with Ex-Fuzzy Demo

This notebook demonstrates a classification computation example using T1 Fuzzy Sets (T1-FS) and shows how to optimize fuzzy rules using a genetic algorithm (GA) Then, we will save the classifier in a text file that we can reuse afterwards. 

#### Authors:
* Javier Fumanal Idocin - University of Essex
* Javier Andreu-Perez - University of Essex

### Requirements
To run this notebook, ensure you have the following packages installed:
- `pandas`
- `scikit-learn`
- `ex_fuzzy` (a custom fuzzy logic package)


In [1]:
# Import necessary libraries
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
import sys


### Setting Up Paths
In case the custom fuzzy logic package (`ex_fuzzy`) is not installed, we add the path to the package manually, ensuring it can be accessed. We handle different possible directory structures.

In [2]:
# Add paths to fuzzy logic package
# Adjust paths as necessary
sys.path.append('./ex_fuzzy/')
sys.path.append('./ex_fuzzy/ex_fuzzy/')

# For launching from a different folder structure (optional)
sys.path.append('../ex_fuzzy/')
sys.path.append('../ex_fuzzy/ex_fuzzy/')


### Importing Custom Fuzzy Logic Modules
Here we import several modules from the custom fuzzy logic package `ex_fuzzy`.

In [3]:
# Import custom fuzzy logic modules
import ex_fuzzy.fuzzy_sets as fs
import ex_fuzzy.evolutionary_fit as GA
import ex_fuzzy.utils as utils
import ex_fuzzy.eval_tools as eval_tools
import ex_fuzzy.persistence as persistence
import ex_fuzzy.vis_rules as vis_rules


### Loading the Iris Dataset
We use the Iris dataset to demonstrate fuzzy set classification. The Iris dataset consists of three classes of iris plants, with 50 samples each, and four features.

In [4]:
# Load Iris dataset and convert it to a DataFrame
iris = datasets.load_iris()
X = pd.DataFrame(iris.data, columns=iris.feature_names)
y = iris.target
n_linguistic_variables = 5  # Define the number of linguistic variables


### Splitting Data into Training and Test Sets
We split the data into a training set (67%) and a test set (33%) to evaluate the model's performance.

In [5]:
# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)


### Defining Fuzzy Sets
We define T1 fuzzy sets and compute fuzzy partitions using the `construct_partitions` function. This generates fuzzy partitions for each feature in the dataset.

In [6]:
# Define the fuzzy set type and compute fuzzy partitions
fz_type_studied = fs.FUZZY_SETS.t1  # T1 fuzzy sets
precomputed_partitions = utils.construct_partitions(X, fz_type_studied, n_partitions=n_linguistic_variables)


### Training a Fuzzy Rules Classifier
We use the `BaseFuzzyRulesClassifier` class from the GA module to train a fuzzy rules classifier. The classifier is trained on the training set for 20 generations of genetic algorithm evolution.

In [7]:
# Train the fuzzy rules classifier
fl_classifier = GA.BaseFuzzyRulesClassifier(nRules=10, 
                                           linguistic_variables=precomputed_partitions, 
                                           nAnts=3, 
                                           n_linguistic_variables=n_linguistic_variables, 
                                           fuzzy_type=fz_type_studied, 
                                           verbose=True, 
                                           tolerance=0.01, 
                                           runner=1, 
                                           ds_mode=2, 
                                           fuzzy_modifiers=False)
fl_classifier.fit(X_train, y_train, n_gen=20)


### Evaluating the Fuzzy Model
We evaluate the performance of the fuzzy model and print the generated rules. The function `eval_fuzzy_model` is used for this evaluation.

In [8]:
# Evaluate the trained fuzzy model
str_rules = eval_tools.eval_fuzzy_model(fl_classifier, X_train, y_train, X_test, y_test, 
                                      plot_rules=False, 
                                      print_rules=True, 
                                      plot_partitions=False, 
                                      return_rules=True)


### Saving Fuzzy Rules and Partitions
The fuzzy rules and partitions are saved to plain text files for later use.

In [9]:
# Save the generated fuzzy rules to a file
with open('iris_rules.txt', 'w') as f:
    f.write(str_rules)

# Save the fuzzy partitions to a file
with open('iris_partitions.txt', 'w') as f:
    str_partitions = persistence.save_fuzzy_variables(precomputed_partitions)
    f.write(str_partitions)


### Loading Saved Rules and Partitions
We load the previously saved rules and partitions from the text files, which can then be used to restore and evaluate the model.

In [10]:
# Load the saved fuzzy rules from a file
with open('iris_rules.txt', 'r') as f:
    str_rules = f.read()

# Load the saved fuzzy partitions from a file
with open('iris_partitions.txt', 'r') as f:
    loaded_partitions = persistence.load_fuzzy_variables(f.read())


### Persistence of the Fuzzy Model
We can restore the master rule base from the saved rules and partitions. The restored model is then evaluated again.

In [11]:
# Restore the master rule base from the loaded rules and partitions
mrule_base = persistence.load_fuzzy_rules(str_rules, loaded_partitions)

# Create a new fuzzy classifier using the restored rules
fl_classifier2 = GA.BaseFuzzyRulesClassifier(precomputed_rules=mrule_base, ds_mode=2, allow_unknown=False)

# Evaluate the restored fuzzy model
str_rules = eval_tools.eval_fuzzy_model(fl_classifier2, X_train, y_train, X_test, y_test, 
                                      plot_rules=True, 
                                      print_rules=True, 
                                      plot_partitions=True, 
                                      return_rules=True)

print('Done')
